In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import datetime
import finplot as fplt

from ta.trend import ADXIndicator
from ta.trend import IchimokuIndicator

In [21]:
start = datetime.datetime(2009,12,4)#26 day before start year for make indicator
end = datetime.datetime(2011,1,1)
st=start.strftime("%Y")
ed=end.strftime("%Y")

In [22]:
stock = 'KO'
df = web.DataReader(stock,'yahoo',start,end)
# df = df.drop(['Open','High','Low','Volume','Adj Close'],axis=1)
# pd.set_option('display.max_columns', None)

In [23]:
df.to_csv('Ground Truth file/'+stock+'('+str(int(st)+1)+','+ed+')'+'.csv')

In [24]:
#4 indicator MACD, RSI, Stochastic, Adx
#MACD
df['EWMA-5'] = df['Close'].ewm(span=5).mean()
df['EWMA-12'] = df['Close'].ewm(span=12).mean()
df['EWMA-26'] = df['Close'].ewm(span=26).mean()
df['MACD'] = df['EWMA-12'] - df['EWMA-26']
df['SIGNAL LINE'] = df['MACD'].ewm(span=9).mean() 

#RSI
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()        
    up_chg = 0 * diff
    down_chg = 0 * diff
    up_chg[diff > 0] = diff[ diff>0 ]
    down_chg[diff < 0] = diff[ diff < 0 ]
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi
df['RSI-7'] = computeRSI(df['Close'],12)
df['RSI-14'] = computeRSI(df['Close'], 26)

#DMI
adxi = ADXIndicator(df['High'],df['Low'],df['Close'],14,False)
df['+DI'] = adxi.adx_pos()
df['-DI'] = adxi.adx_neg()
df['adx'] = adxi.adx()

#STOCHASTIC Slow-fast
# Fast Stochastic
low_min  = df['Low'].rolling( window = 9 ).min()
high_max = df['High'].rolling( window = 9 ).max()
df['%K'] = (100*(df['Close']-low_min)/(high_max-low_min))
df['%D']=df['%K'].ewm(span=5).mean() 

#SLOW stochastic
df['%Ks'] = df['%D']
df['%Ds']=df['%Ks'].ewm(span=5).mean() 

#Ichimoku
ichi = IchimokuIndicator(df['High'],df['Low'],n1=9,n2=26,n3=52,visual=True,fillna=True)
df['ichi_a'] = ichi.ichimoku_a()
df['ichi_b'] = ichi.ichimoku_b()
df['ichi_base'] = ichi.ichimoku_base_line()
df['ichi_con']=ichi.ichimoku_conversion_line()

df['70'] = 70
df['30'] = 30
df['80'] = 80
df['20'] = 20

C:\Users\jame_\anaconda3\lib\site-packages\ta\trend.py:608: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
C:\Users\jame_\anaconda3\lib\site-packages\ta\trend.py:612: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


In [ ]:
symbol = stock
ax,ax2,ax3,ax4 = fplt.create_plot(symbol, rows=4)

#plot1 main
fplt.candlestick_ochl(df[['Open','Close','High','Low']],ax=ax)

fplt.candlestick_ochl(df[['Open','Close','High','Low']],ax=ax)
p0=fplt.plot(df['ichi_a'],ax=ax,legend='ichi_a')
p1=fplt.plot(df['ichi_b'],ax=ax,legend='ichi_b')
# fplt.plot(df['ichi_base'],ax=ax,legend='base')
# fplt.plot(df['ichi_con'],ax=ax,legend='convo')
fplt.fill_between( p0, p1, color='#bbb')

#MACD
fplt.plot(df['MACD'],ax=ax2,legend='MACD')
fplt.plot(df['SIGNAL LINE'],ax=ax2,legend='SIGNAL LINE')

#RSI
fplt.plot(df['RSI-7'],ax=ax3,legend='RSI-7')
fplt.plot(df['RSI-14'],ax=ax3,legend='RSI-14')
y1 = fplt.plot(df['70'], ax=ax3,color='r')
y2 = fplt.plot(df['30'], ax=ax3,color='r')
fplt.fill_between(y1, y2, color='#bbb')

#DMI
fplt.plot(df['+DI'],ax=ax4,legend='+DI')
fplt.plot(df['-DI'],ax=ax4,legend='-DI')


fplt.show()